In [260]:
import numpy as np
import math
import pandas as pd
import itertools
import random

In [261]:
#GENERATE X, y
seed = 0
random.seed(seed)
n =	10
m =	3
C =	int(((n * 60)/m)*2)  # int(((n * 60)/m)*1 oppure *1.5 oppure *2)
Tmax =	100  # 30 or 100
name =	'I2_N' + str(n)  + "_T" + str(Tmax) + "_C" + str(C) +'_' + str(seed)
LT =	3
LS =	5
print(f"base capacity = {C}")
p =	10

Qmax =	10000
M =	10000
service_time =	0
D =	50
delta = 10
Depot	= 'Central'  # 'LowerLeft'

minx = 0
miny = 0
maxx = 10
maxy = 10


df_param = pd.DataFrame()
df_param['param'] = ['name', 'n', 'LT', 'm', 'LS', 'C', 'p', 'Tmax', 'Qmax', 'M', 'service_time', 'D', 'Depot', 'seed']
df_param['value'] = [name, n, LT, m, LS, C, p, Tmax, Qmax, M, 0, D, Depot, seed]

df_time_windows = pd.DataFrame()
df_time_windows['t'] = [1,2,3]
if Tmax ==	30:
    df_time_windows['e'] =  [0, 10, 20]
    df_time_windows['l'] = [10, 20, 30]
else:
    df_time_windows['e'] =  [0, 15, 30]
    df_time_windows['l'] = [15, 30, 45]

df_edges = pd.DataFrame()
df_edges['i'] = [0,    0,    1, 1, 2, 2]
df_edges['j'] = [1,    2,    0, 2, 1, 0]
df_edges['c'] = [10.2, 11.2, 1, 2, 3, 4]
df_edges['t'] = [10.2, 11.2, 1, 2, 3, 4]




base capacity = 400


In [262]:
# GENERATE NODES AND LOCATIONS
rndy = [round(random.random()*(maxy - miny) + miny,2) for i in range(n)]
rndx = [round(random.random()*(maxx - minx) + minx,2) for i in range(n)]

if Depot == 'LowerLeft':
    rndy = [0] + rndy
    rndx = [0] + rndx
if Depot == 'Central':
    rndy = [5] + rndy
    rndx = [5] + rndx
df_nodes = pd.DataFrame()
df_nodes['i'] = [i for i in range(n + 1)]
df_nodes['x'] = rndx
df_nodes['y'] = rndy
df_nodes['service_time'] = [service_time for i in range(n+1)]


In [263]:
# GENERATING PATTERNS FOR CUSTOMER DEMAND AND DELTA
patternCustomersDict = {
    1: {'demand' : 'low',
        'demand_value' : 0.5,
        'delta'  : ['high', 'low', 'low'],
        'delta_value'  : [1.5, 0.5, 0.5]},
    2: {'demand' : 'average',
        'demand_value' : 1,
        'delta'  : ['average', 'high', 'low'],
        'delta_value'  : [1, 1.5, 0.5]},
    3: {'demand' : 'high',
        'demand_value' : 1.5,
        'delta'  : ['low', 'low', 'average'],
        'delta_value'  : [0.5, 0.5, 1]}

}

maxDemand = D*1.5 
minDemand = D*0.5
maxdelta = delta*1.5
mindelta = delta*0.5


In [264]:
#GENERATING DEMAND AND DELTA

#rnd_minDelta = [round(random.random()*mindelta,2)  for i in range(LS*LT*n)]
#rnd_maxDelta = [round(random.random()*maxdelta, 2) for i in range(LS*LT*n)]
#rndDelta = [round(random.random()*(rnd_maxDelta[i] - rnd_minDelta[i]) + rnd_minDelta[i],2) for i in range(LS*LT*n)]
rndDelta = [round(random.random()*(maxdelta - mindelta) + mindelta,2) for i in range(LS*LT*n)]
rndDemand = [round(random.random()*(maxDemand - minDemand) + minDemand,2) for i in range(n)]
i_d = 0
for i in range(n):
    PatternCustomers =	random.randint(1, len(patternCustomersDict))
    pc = patternCustomersDict[PatternCustomers]
    rndDemand[i] = rndDemand[i] * pc['demand_value']
    for t in range (LT):
        delta_current = pc['delta_value'][t]
        for s in range (LS):
            rndDelta[i_d] = rndDelta[i_d]* delta_current
            i_d += 1

indices = [(i, t, s) for (i,t,s) in itertools.product(range(1, n+1), range(1, LT+1), range(1, LS+1)) ]
df_delta = pd.DataFrame()
df_demand = pd.DataFrame()
df_delta['j'] = [i[0] for i in indices]
df_delta['t'] = [i[1] for i  in indices]
df_delta['s'] = [i[2] for i in indices]
df_delta['delta'] = rndDelta
df_demand['j'] = [j for j in range(1,n+1)]
df_demand['d'] = rndDemand

In [265]:
# writing output

filename = 'instances/' + name +  '.xlsx'
with pd.ExcelWriter(filename) as writer:
    df_param.to_excel(writer, sheet_name='params', index=None)
    df_nodes.to_excel(writer, sheet_name='nodes', index=None)
    #df_delta.to_excel(writer, sheet_name='delta', index=None)
    df_edges.to_excel(writer, sheet_name='edges', index=None)
    df_time_windows.to_excel(writer, sheet_name='time_windows', index=None)
    df_demand.to_excel(writer, sheet_name='demand', index=None)
    df_delta.to_excel(writer, sheet_name='delta', index=None)



In [266]:
# COMPUTNG THE AVERAGE SCENARIO
dfmean = df_delta.groupby(['j', 't']).agg({'delta' : 'mean'})
dfmean = dfmean.reset_index()
#dfmean['s'] = [1 for i in len(dfmean)]
dfmean.insert(loc=2, column='s', value=[1 for i in range(n*LT)])
dfmean

,j,t,s,delta
0,1,1,1,5.595
1,1,2,1,5.526
2,1,3,1,10.912
3,2,1,1,5.126
4,2,2,1,4.432
5,2,3,1,10.672
6,3,1,1,13.977
7,3,2,1,5.347
8,3,3,1,6.009
9,4,1,1,4.792


In [267]:
# writing average scenario
name_mean = name + "_mean"
df_param.loc[df_param['param'] == 'name', 'value'] = name_mean
df_param.loc[df_param['param'] == 'LS', 'value'] = 1
filename = 'instances/' + name_mean +  '.xlsx'
with pd.ExcelWriter(filename) as writer:
    df_param.to_excel(writer, sheet_name='params', index=None)
    df_nodes.to_excel(writer, sheet_name='nodes', index=None)
    #df_delta.to_excel(writer, sheet_name='delta', index=None)
    df_edges.to_excel(writer, sheet_name='edges', index=None)
    df_time_windows.to_excel(writer, sheet_name='time_windows', index=None)
    df_demand.to_excel(writer, sheet_name='demand', index=None)
    dfmean.to_excel(writer, sheet_name='delta', index=None)
    #df_delta.to_excel(writer, sheet_name='delta', index=None)


#dfmean.to_excel('scenario_mean_S4.xlsx', sheet_name='delta', index=None)